In [333]:
import numpy as np

In [334]:
# Row Echelon Form

In [457]:
def rowechelonform(M, verbose=False):

    M = M.copy()
    M = M.astype('float')
    def sortTransform(M, verbose = verbose):
        
        newM = M.copy()
        
        
        
        for k in reversed(range(newM.shape[1])):
            # Create a boolean matrix, where 1 is values above 0.
            boolM = np.where(np.isclose(newM, 0), 0 ,1)
            # Sort the matrix by the boolean matrix, this preserves the order as best as possible, only switching the rows when there is a non-zero value below a zero.
            newIndeces = np.argsort(-boolM[:,k])
            # Sort the matrix
            newM = newM[newIndeces]
            
        return newM

    def refTransform(M, verbose = verbose):

        if M.shape[0] == 1 and M.shape[1] == 1:
            print(f"Matrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            M = M / M if M > 1 else M
            return M

        elif M.shape[0] == 0 or M.shape[1] == 0:
            print(f"Matrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            return M
        
        elif np.all(M==0):
            print(f"Matrix is is all zeros") if verbose == True else None
            return M

        else:
            
            #Sort Matrix
            A = sortTransform(M) if np.nonzero(M[:,0])[0].size < len(M[:,0]) else M.copy()
            print(f"Sorting Matrix \n {str(A)}") if verbose == True else None
            # Divide row by its first non-zero element
            for row in range(M.shape[0]):
                print(f"Dividing row {row} ({A[row]}) by {A[row,0]} to get {A[row] / A[row,0] if A[row,0] else A[row]}") if verbose == True else None
                A[row] = A[row] / A[row,0] if A[row,0] else A[row]
                print(f"The result is {A}") if verbose == True else None
            # Subtract the row from all other rows
            A[1:] -= A[0] * A[1:,0:1]
            print(f"Subtracting row {0} from all other rows.\n the result is\n {str(A)}") if verbose == True else None
            # Check if there are any other non-zero elements below the first row  
            k = 1 if np.nonzero(A[:,0])[0].size > 0 else 0 
            # If there are, repeat the process by taking a subset of the matrix, k rows below the first (determined by whether there are non-zero elements below the first row)
            B = A[k:,1:]
            print(f"Sending the subset of the matrix\n {str(B)}") if verbose == True else None
            # Recursively call the function
            B = refTransform(B)
            # Update the matrix
            A[k:,1:] = B
            print(f"The resulting matrix is\n {str(A)}") if verbose == True else None
            
            return A
        
    newM = refTransform(M)
    
    return newM

In [336]:
# Back Substitution

In [337]:
def backsubstitution(M, verbose = False):
    A = M.copy()
    numrows = A.shape[0]
    if not numrows == 1:
        #
        for row in reversed(range(numrows-1)):

            print("*"*75) if verbose == True else None
            
            col_index = np.nonzero(A[-1,:])[0][0]
            
            value = M[row,col_index]
            
            print(f"\nWe multiply {value} against {M[-1]} to get {value * M[-1]}") if verbose == True else None
            
            print(f"\nWe substract {value * M[-1]} from {M[row]}") if verbose == True else None
           
            A[row] = M[row] - value * M[-1] # Using M to preserve initial value
            
            print(f"\nThe result is: {A[row]}") if verbose == True else None

        B = A[:-1,:]

        print("*"*75) if verbose == True else None
        
        print(f"\nSending this matrix into recursion: \n{B}\n...to restart the process and assess against a new pivot.") if verbose == True else None
        
        A[:-1,:] = backsubstitution(B,verbose=verbose)


    else:
        print('\nReached first row') if verbose == True else None
        return A

    #final check

    return A




In [362]:
def gaussianelimination(M,verbose=False):
    A = M.copy()
    n = np.min([A.shape[0], A.shape[1]])
    
    if np.linalg.det(M[:n,:n]) == 0:
        print('Singular Matrix')
        return A
    else:
        A = rowechelonform(A,verbose=verbose)
        if not isinstance(A,str):
            A = backsubstitution(A,verbose=verbose)
        return A

In [459]:
M = np.array([[1,-1,.5,.5],[0,1,1,-1],[0,0,1,-1]])
gaussianelimination(M)

array([[ 1.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -1.]])

In [460]:
M = np.array([[-2,0,-1,2],[4,0,1,0],[6,0,1,2]])
gaussianelimination(M,verbose=True)

Singular Matrix


array([[-2,  0, -1,  2],
       [ 4,  0,  1,  0],
       [ 6,  0,  1,  2]])

In [458]:
M = np.array([[-2,1,-1,1],[3,4,2,13],[1,-5,-2,0]])
gaussianelimination(M,verbose=True)

Sorting Matrix 
 [[-2.  1. -1.  1.]
 [ 3.  4.  2. 13.]
 [ 1. -5. -2.  0.]]
Dividing row 0 ([-2.  1. -1.  1.]) by -2.0 to get [ 1.  -0.5  0.5 -0.5]
The result is [[ 1.  -0.5  0.5 -0.5]
 [ 3.   4.   2.  13. ]
 [ 1.  -5.  -2.   0. ]]
Dividing row 1 ([ 3.  4.  2. 13.]) by 3.0 to get [1.         1.33333333 0.66666667 4.33333333]
The result is [[ 1.         -0.5         0.5        -0.5       ]
 [ 1.          1.33333333  0.66666667  4.33333333]
 [ 1.         -5.         -2.          0.        ]]
Dividing row 2 ([ 1. -5. -2.  0.]) by 1.0 to get [ 1. -5. -2.  0.]
The result is [[ 1.         -0.5         0.5        -0.5       ]
 [ 1.          1.33333333  0.66666667  4.33333333]
 [ 1.         -5.         -2.          0.        ]]
Subtracting row 0 from all other rows.
 the result is
 [[ 1.         -0.5         0.5        -0.5       ]
 [ 0.          1.83333333  0.16666667  4.83333333]
 [ 0.         -4.5        -2.5         0.5       ]]
Sending the subset of the matrix
 [[ 1.83333333  0.16666667  4

array([[ 1.        ,  0.        ,  0.        ,  4.04347826],
       [ 0.        ,  1.        ,  0.        ,  3.17391304],
       [ 0.        ,  0.        ,  1.        , -5.91304348]])